In [0]:
import tensorflow as tf
import os

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 813 (delta 11), reused 23 (delta 8), pack-reused 782
Receiving objects: 100% (813/813), 2.15 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [3]:
BUCKET = 'gs://gs_colab'
PROJECT = 'dvc'
MODEL = "InceptionResNetV2" #@param ["InceptionResNetV2", "ResNet50", "ResNet50V2", "InceptionV3", "MobileNetV2", "Xception"]

BATCH_SIZE = 128 #@param ["128", "256", "512"] {type:"raw"}
EPOCHS = 1 #@param {type:"slider", min:0, max:50, step:1}
LEARNING_RATE = 0.001 #@param ["0.001", "0.01"] {type:"raw"}

In [4]:
fw.colab_utils.setup_gcs()

In [5]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [6]:
base_model = fw.keras_models.get_model(MODEL, BUCKET)

In [7]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.DVC, os.path.join('.', PROJECT))
train_dir_local = os.path.join(data_dir_local, 'dogscats/train')
valid_dir_local = os.path.join(data_dir_local, 'dogscats/valid')

In [8]:
train_fn = os.path.join(data_dir, 'train.tfrec')
valid_fn = os.path.join(data_dir, 'valid.tfrec')
paths_train, y_train, labels_train = fw.data.data_dir_tfrecord(train_dir_local, train_fn)
paths_valid, y_valid, labels_valid = fw.data.data_dir_tfrecord(valid_dir_local, valid_fn)
n_train, n_valid = len(y_train), len(y_valid)
n_classes = len(labels_train)

In [9]:
def build_nn(c=256):
  base = base_model.model_func()
  base.trainable = False  
  model = fw.Sequential()
  model.add(base)
  model.add(tf.keras.layers.Flatten())
  model.add(fw.layers.DenseBN(c))
  model.add(fw.layers.Classifier(n_classes))
  return model

In [10]:
total_steps = n_train // BATCH_SIZE * EPOCHS

In [11]:
lr_func = fw.train.cosine_lr(init_lr=LEARNING_RATE, total_steps=total_steps)
fw.plt.plot_lr_func(lr_func, total_steps=total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.train.get_clf_model_func(build_nn, opt_func)

In [13]:
parser_train = fw.data.get_tfexample_image_parser(base_model.img_size, base_model.img_size, training=True, normalizer=base_model.normalizer)
parser_eval = fw.data.get_tfexample_image_parser(base_model.img_size, base_model.img_size, training=False, normalizer=base_model.normalizer)

train_input_func = lambda params: fw.data.tfrecord_ds(train_fn, parser_train,
  params['batch_size'], training=True)
valid_input_func = lambda params: fw.data.tfrecord_ds(valid_fn, parser_eval, 
  params['batch_size'], training=False)

In [14]:
est = fw.train.get_tpu_estimator(n_train // BATCH_SIZE,  model_func, work_dir, 
  base_model.weight_dir, base_model.weight_vars, trn_bs=BATCH_SIZE, 
  val_bs = n_valid)

est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/dvc/2019-04-21-16:33:14', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.109.71.18:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa831830898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.109.71.18:8470', '_evaluation_master': 'grpc://10.109.71.18:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=179, num_shards=None, num

In [15]:
result = est.evaluate(valid_input_func, steps=1)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-21T16:36:30Z
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/dvc/2019-04-21-16:33:14/model.ckpt-179
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 7 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:

In [16]:
print(f'Test results: accuracy={result["accuracy"] * 100: .2f}%, loss={result["loss"]: .2f}.')

Test results: accuracy= 99.35%, loss= 0.02.


In [17]:
fw.io.create_clean_dir(work_dir)